In [ ]:
import numpy as np 
import pandas as pd 
import random, math, time, os 
import polars as pl 
 

In [2]:
TRAIN_DIRECTORY = "./csv_files/train/"
TEST_DIRECTORY = "./csv_files/test/"

df_train_base = pl.read_csv(TRAIN_DIRECTORY + "train_base.csv")
df_test_base = pl.read_csv(TEST_DIRECTORY + "test_base.csv")
df_static_0_0 = pl.read_csv(TRAIN_DIRECTORY + "train_static_0_0.csv")

In [3]:
def set_table_dtypes(df: pl.DataFrame) -> pl.DataFrame:
    # implement here all desired dtypes for tables
    # the following is just an example
    for col in df.columns:
        # last letter of column name will help you determine the type
        if col[-1] in ("P", "A"):
            df = df.with_columns(pl.col(col).cast(pl.Float64).alias(col))

    return df

In [ ]:
train_basetable = pl.read_csv(TRAIN_DIRECTORY + "train_base.csv")

'train_static = pl.concat(\n    [\n        pl.read_csv(TRAIN_DIRECTORY + "train_static_0_0.csv").pipe(set_table_dtypes),\n        pl.read_csv(TRAIN_DIRECTORY + "train_static_0_1.csv").pipe(set_table_dtypes),\n    ],\n    how="vertical_relaxed",\n)\ntrain_static_cb = pl.read_csv(TRAIN_DIRECTORY + "train_static_cb_0.csv").pipe(set_table_dtypes)\ntrain_person_1 = pl.read_csv(TRAIN_DIRECTORY + "train_person_1.csv").pipe(set_table_dtypes) \ntrain_credit_bureau_b_2 = pl.read_csv(TRAIN_DIRECTORY + "train_credit_bureau_b_2.csv").pipe(set_table_dtypes) '

In [5]:
# Gathering training data! 
# Focusing on depth 0
train_basetable = pl.read_csv(TRAIN_DIRECTORY + "train_base.csv")
column_set = set() 
for root, dirs, file in os.walk(TRAIN_DIRECTORY):
    for file_name in file:
        if "_static_" in file_name:
            df_temp = pl.read_csv(TRAIN_DIRECTORY + file_name)
            train_basetable = train_basetable.join(df_temp, on="case_id", how="left")

In [6]:
case_id_defaulted = train_basetable.filter(pl.col("target") == 1)["case_id"]
case_id_not_defaulted = train_basetable.filter(pl.col("target") == 0)["case_id"]

In [7]:
print(train_basetable)

shape: (1_526_659, 391)
┌─────────┬────────────┬────────┬──────────┬───┬────────────┬────────────┬────────────┬────────────┐
│ case_id ┆ date_decis ┆ MONTH  ┆ WEEK_NUM ┆ … ┆ riskassesm ┆ riskassesm ┆ secondquar ┆ thirdquart │
│ ---     ┆ ion        ┆ ---    ┆ ---      ┆   ┆ ent_302T   ┆ ent_940T   ┆ ter_766L   ┆ er_1082L   │
│ i64     ┆ ---        ┆ i64    ┆ i64      ┆   ┆ ---        ┆ ---        ┆ ---        ┆ ---        │
│         ┆ str        ┆        ┆          ┆   ┆ str        ┆ f64        ┆ f64        ┆ f64        │
╞═════════╪════════════╪════════╪══════════╪═══╪════════════╪════════════╪════════════╪════════════╡
│ 0       ┆ 2019-01-03 ┆ 201901 ┆ 0        ┆ … ┆ null       ┆ null       ┆ null       ┆ null       │
│ 1       ┆ 2019-01-03 ┆ 201901 ┆ 0        ┆ … ┆ null       ┆ null       ┆ null       ┆ null       │
│ 2       ┆ 2019-01-04 ┆ 201901 ┆ 0        ┆ … ┆ null       ┆ null       ┆ null       ┆ null       │
│ 3       ┆ 2019-01-03 ┆ 201901 ┆ 0        ┆ … ┆ null       ┆ null 

In [30]:
eliminate_column_set = set()
for column in train_basetable.columns:
    if train_basetable[column].null_count() / len(train_basetable[column]) >= 0.8:
        eliminate_column_set.add(column)                                            


In [35]:
for column in eliminate_column_set:
    train_basetable = train_basetable.drop(column)

In [36]:
print(train_basetable)

shape: (1_526_659, 287)
┌─────────┬────────────┬────────┬──────────┬───┬────────────┬────────────┬────────────┬────────────┐
│ case_id ┆ date_decis ┆ MONTH  ┆ WEEK_NUM ┆ … ┆ responseda ┆ responseda ┆ secondquar ┆ thirdquart │
│ ---     ┆ ion        ┆ ---    ┆ ---      ┆   ┆ te_1012D   ┆ te_4527233 ┆ ter_766L   ┆ er_1082L   │
│ i64     ┆ ---        ┆ i64    ┆ i64      ┆   ┆ ---        ┆ D          ┆ ---        ┆ ---        │
│         ┆ str        ┆        ┆          ┆   ┆ str        ┆ ---        ┆ f64        ┆ f64        │
│         ┆            ┆        ┆          ┆   ┆            ┆ str        ┆            ┆            │
╞═════════╪════════════╪════════╪══════════╪═══╪════════════╪════════════╪════════════╪════════════╡
│ 0       ┆ 2019-01-03 ┆ 201901 ┆ 0        ┆ … ┆ null       ┆ null       ┆ null       ┆ null       │
│ 1       ┆ 2019-01-03 ┆ 201901 ┆ 0        ┆ … ┆ null       ┆ null       ┆ null       ┆ null       │
│ 2       ┆ 2019-01-04 ┆ 201901 ┆ 0        ┆ … ┆ null       ┆ null 